In [1]:
import config
import geopandas as gpd
import json
import networkx as nx
import numpy as np
import os
import osmnx as ox
import pandas as pd
import time

ox.config(use_cache=True,
          log_file=True,
          log_console=True,
          log_filename='calculate-tracts',
          cache_folder=config.tracts_cache_folder)

print(ox.__version__)
print(nx.__version__)

0.8.1
2.1


In [2]:
graphml_folder = config.tracts_graphml_folder
tracts_folder = 'input_data/tracts' #tiger tract shapefiles
stats_folder = config.tracts_stats_folder

In [3]:
tracts = []
for state_folder in os.listdir(graphml_folder):
    for tract_file in os.listdir('{}/{}'.format(graphml_folder, state_folder)):

        data = {}
        data['state_folder'] = state_folder
        data['state_fips'] = state_folder.split('_')[0]
        data['state'] = state_folder.split('_')[1]
        data['tract_file'] = tract_file
        data['geoid'] = tract_file.replace('.graphml', '')
        tracts.append(data)

df = pd.DataFrame(tracts)

## Get land area data from shapefiles and merge into DataFrame

In [4]:
# load each state shapefile and get the geoid and aland for each tract row
gdf = gpd.GeoDataFrame()
for state_fips in df['state_fips'].unique():
    path = '{}/tl_2017_{}_tract'.format(tracts_folder, state_fips)
    gdf = gdf.append(gpd.read_file(path)[['GEOID', 'ALAND']])

# merge aland values into dataframe, on geoid
gdf = gdf.rename(columns=str.lower)
df = pd.merge(df, gdf, how='left', on='geoid')

## Load graph and calculate stats for each tract

In [5]:
def load_graph_get_stats(row):
    
    try:
        start_time = time.time()
        folder = '{}/{}'.format(graphml_folder, row['state_folder'])
        G = ox.load_graphml(filename=row['tract_file'], folder=folder)
        
        # if tract has no land area, set area to null to avoid div by zero
        area = row['aland'] if row['aland'] > 0 else np.nan        
        stats = ox.basic_stats(G, area=area)
        
        # unpack k-counts and k-proportion dicts into individiual keys:values
        for k, count in stats['streets_per_node_counts'].items():
            stats['int_{}_streets_count'.format(k)] = count
        for k, proportion in stats['streets_per_node_proportion'].items():
            stats['int_{}_streets_prop'.format(k)] = proportion
            
        # calculate/drop the extended stats that have values per node
        extended_stats = ox.extended_stats(G)
        se = pd.Series(extended_stats)
        se = se.drop(['avg_neighbor_degree', 'avg_weighted_neighbor_degree', 'clustering_coefficient',
                      'clustering_coefficient_weighted', 'degree_centrality', 'pagerank'])
        extended_stats_clean = se.to_dict()
        
        for key in extended_stats_clean:
            stats[key] = extended_stats_clean[key]
        
        stats['area_km'] = row['aland'] / 1e6
        stats['state'] = row['state']
        stats['geoid'] = row['geoid']
        stats['area'] = row['aland']
        stats['time'] = time.time()-start_time
        
        return pd.Series(stats)

    except Exception as e:
        print('{}, {} failed: {}'.format(row['geoid'], row['state'], e))
        return pd.Series()

In [6]:
#sample = list(range(0, len(df), int(len(df)/100)))
#stats = df.iloc[sample].apply(load_graph_get_stats, axis=1)
stats_temp = df.apply(load_graph_get_stats, axis=1)
stats_temp.shape

g:\geoff\dropbox\documents\school\phd\projects\code\osmnx-repos\osmnx\osmnx\stats.py:201: RuntimeWarning: divide by zero encountered in double_scalars
  circuity_avg = edge_length_total / gc_distances.sum()


06029004601, CA failed: float division by zero
17097863006, IL failed: float division by zero
26033980300, MI failed: float division by zero


(72663, 66)

In [7]:
stats_temp['time'].sum()

15758.280586481094

## Clean up the dataframe

In [8]:
stats = stats_temp.copy()

In [9]:
# stuff to drop
cols_to_drop = ['area', 'time', 'streets_per_node_counts', 'streets_per_node_proportion', 
                'pagerank_max_node', 'pagerank_min_node', 'clean_intersection_count',
                'clean_intersection_density_km']

In [10]:
cols_to_rename = {}
for col in stats.columns:
    if 'int_' in col:
        n = col.split('_')[1]
        if n not in ['1', '3', '4']:
            cols_to_drop.append(col)
        else:
            suffix = 'count' if 'count' in col else 'proportion'
            cols_to_rename[col] = 'intersect_{}way_{}'.format(n, suffix)
            
stats = stats.drop(cols_to_drop, axis=1)

In [11]:
# rename these to friendlier names
cols_to_rename['clustering_coefficient_avg'] = 'cluster_coeff_avg'
cols_to_rename['clustering_coefficient_weighted_avg'] = 'cluster_coeff_weighted_avg'
cols_to_rename['intersection_density_km'] = 'intersect_density_km'
cols_to_rename['intersect_1way_count'] = 'dead_end_count'
cols_to_rename['intersect_1way_proportion'] = 'dead_end_proportion'
cols_to_rename['m'] = 'edge_count'
cols_to_rename['n'] = 'node_count'
stats = stats.rename(columns=cols_to_rename)
stats = stats.rename(columns=cols_to_rename)

In [12]:
# drop anything lacking a GEOID
stats = stats.dropna(subset=['geoid'])

In [13]:
# make these integers
cols_int = ['intersection_count', 'edge_length_total', 'edge_count', 'node_count', 'street_segments_count']
stats[cols_int] = stats[cols_int].astype(np.int64)

In [14]:
# make state, geoid at left of df
cols = stats.columns.tolist()
cols.insert(0, cols.pop(cols.index('geoid')))
cols.insert(1, cols.pop(cols.index('state')))
stats = stats.reindex(columns=cols)

## View the results

In [15]:
stats.shape

(72660, 32)

In [16]:
stats.columns

Index(['geoid', 'state', 'area_km', 'avg_neighbor_degree_avg',
       'avg_weighted_neighbor_degree_avg', 'circuity_avg', 'cluster_coeff_avg',
       'cluster_coeff_weighted_avg', 'degree_centrality_avg',
       'edge_density_km', 'edge_length_avg', 'edge_length_total',
       'dead_end_count', 'dead_end_proportion', 'intersect_3way_count',
       'intersect_3way_proportion', 'intersect_4way_count',
       'intersect_4way_proportion', 'intersection_count',
       'intersect_density_km', 'k_avg', 'edge_count', 'node_count',
       'node_density_km', 'pagerank_max', 'pagerank_min',
       'self_loop_proportion', 'street_density_km', 'street_length_avg',
       'street_length_total', 'street_segments_count', 'streets_per_node_avg'],
      dtype='object')

In [17]:
stats.head()

,geoid,state,area_km,avg_neighbor_degree_avg,avg_weighted_neighbor_degree_avg,circuity_avg,cluster_coeff_avg,cluster_coeff_weighted_avg,degree_centrality_avg,edge_density_km,...,node_count,node_density_km,pagerank_max,pagerank_min,self_loop_proportion,street_density_km,street_length_avg,street_length_total,street_segments_count,streets_per_node_avg
0,01001020100,AL,9.817812,2.580952,0.043010,1.059221,0.034286,0.003515,0.025419,7091.146378,...,175,17.824745,0.014839,0.000872,0.005168,3730.982626,181.337059,36630.086,202,2.531429
1,01001020200,AL,3.325679,2.844282,0.030708,1.026998,0.022628,0.003143,0.036604,15896.487905,...,137,41.194595,0.014139,0.001465,0.000000,8024.587159,155.158145,26687.201,172,2.759124
2,01001020300,AL,5.349273,2.691011,0.031611,1.048203,0.048689,0.005347,0.025900,13352.252166,...,178,33.275550,0.016649,0.000843,0.009804,7078.691254,172.117509,37865.852,220,2.702247
3,01001020400,AL,6.384276,2.930931,0.026669,1.056506,0.039790,0.006313,0.023684,14033.430886,...,222,34.772933,0.008728,0.000685,0.000000,7249.537927,152.749343,46283.051,303,2.864865
4,01001020500,AL,11.408873,2.779865,0.029602,1.093550,0.044416,0.006136,0.012090,12370.638099,...,394,34.534524,0.007784,0.000382,0.014957,6576.122725,154.057801,75026.149,487,2.538071


## Save to disk

In [18]:
if not os.path.exists(stats_folder):
    os.makedirs(stats_folder)
output_path = '{}/tracts-stats.csv'.format(stats_folder)
stats.to_csv(output_path, encoding='utf-8', index=False)